# NISQ SDP

Semidefinite programming with NISQ computer. 
Solves the Lovasz-Theta number or a Bell game using a SDP via a quantum computer.

Set model=0 for Lovasz-Theta and model=1 for Bell game.

The classical SDP solves a cost function with constraints.

For NISQ SDP, we use following approach:

We use an ansatz state (either a product state of all zero, or a random state generated via randomized circuit).
The ansatz space of size n_ansatz is generated by applying various Pauli X strings onto the ansatz state.
Then, we decompose the cost matrix and the constraints into a sum of Pauli strings.
We use the ansatz space to measure various overlaps between them usiing the Pauli strings of cost and constraint.
Then, the measured overlaps are used to solve an SDP that gives the solution.
The convergence depends on the n_ansatz parameter.



The code requires qutip as well as cvxpy.

@author: Tobias Haug (github txhaug), Kishor Bharti

Contact at thaug@ic.ac.uk


In [1]:
import qutip as qt

from functools import partial

import operator
from functools import reduce
import numpy as np

import cvxpy as cp

In [2]:
#set parameters
seed=1 #seed of random generator

model=0 #0: Lovasz-Theta number for N=3 qubits with specified graph, 1: Bell game

ansatz_state=0# type of ansatz state. 0: all zero product state 1: randomized quantum circuit composed of y rotations and CNOT gates

#the more anssatz states, the better the convergence
n_ansatz=8 #how many ansatz states are used, upper bounded by 2**n_qubits

verbose_SDP=True #verbose SDP solver

if(model==0): ##Lovasz-Theta number for graph
    n_qubits=3 #number qubits
    ##graph for constraints to solve for Lovasz-Theta number
    graph = np.array([
           [0,1,0,0,1,0,0,1],
           [1,0,1,0,0,1,0,0],
           [0,1,0,1,0,0,1,0],
           [0,0,1,0,1,0,0,1],
           [1,0,0,1,0,1,0,0],
           [0,1,0,0,1,0,1,0],
           [0,0,1,0,0,1,0,1],
           [1,0,0,1,0,0,1,0]])
elif(model==1): #Bell game
    n_qubits=2
    
if(n_ansatz>2**n_qubits):
    raise NameError("Too many ansatz states")
    

In [3]:
def prod(factors):
    return reduce(operator.mul, factors, 1)

def flatten(l):
    return [item for sublist in l for item in sublist]

#tensors operators together 
def genFockOp(op,position,size,levels=2,opdim=0):
    opList=[qt.qeye(levels) for x in range(size-opdim)]
    opList[position]=op
    return qt.tensor(opList)

#get Pauli operator from pauli string
def get_pauli_op(pauli_string):
    pauli_circuit=opId
    for i in range(len(pauli_string)):
        if(pauli_string[i]!=0):
            if(pauli_string[i]==1):
                pauli_circuit=pauli_circuit*opX[i]
            elif(pauli_string[i]==2):
                pauli_circuit=pauli_circuit*opY[i]
            elif(pauli_string[i]==3):
                pauli_circuit=pauli_circuit*opZ[i]

    return pauli_circuit


In [4]:
def numberToBase(n, b,n_qubits):
    if n == 0:
        return np.zeros(n_qubits,dtype=int)
    digits = np.zeros(n_qubits,dtype=int)
    counter=0
    while n:
        digits[counter]=int(n % b)
        n //= b
        counter+=1
    return digits[::-1]
                
def decomposePauli(H):
    """Decompose matrix H into Pauli matrices"""
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    dim_matrix=np.shape(H)[0]
    n_qubits=int(np.log2(dim_matrix))
    if(dim_matrix!=2**n_qubits):
        raise NameError("matrix is not power of 2!")
    hilbertspace=2**n_qubits
    n_paulis=4**n_qubits
    pauli_list=np.zeros([n_paulis,n_qubits],dtype=int)
    for k in range(n_paulis):
        pauli_list[k,:]=numberToBase(k,4,n_qubits)
    weights=np.zeros(n_paulis,dtype=np.complex128)
    for k in range(n_paulis):
        pauli=S[pauli_list[k][0]]

        for n in range(1,n_qubits):
            pauli=np.kron(pauli,S[pauli_list[k][n]])

        #weights[k] = 1/hilbertspace* (np.dot(H.conjugate().transpose(), pauli)).trace()
        weights[k] = 1/hilbertspace* np.dot(pauli,H).trace()

    return pauli_list,weights

In [5]:

#random generator used
rng = np.random.default_rng(seed)

#operators for circuit
levels=2#
opZ=[genFockOp(qt.sigmaz(),i,n_qubits,levels) for i in range(n_qubits)]
opX=[genFockOp(qt.sigmax(),i,n_qubits,levels) for i in range(n_qubits)]
opY=[genFockOp(qt.sigmay(),i,n_qubits,levels) for i in range(n_qubits)]
opId=genFockOp(qt.qeye(levels),0,n_qubits)

  

In [6]:
##define Pauli operators needed for cost function and constraints of NISQ SDP

qt_dims=[[2 for i in range(n_qubits)],[2 for i in range(n_qubits)]] ##dimensions for qutip objects

hilbertspace=2**n_qubits
if(model==0): #set up quantum operators for Lovasz-Theta number
    ##define matrix for cost operator
    cost_op=qt.Qobj(np.ones([hilbertspace,hilbertspace]),dims=qt_dims)
    
    ##define operator for constraints
    constraints_op=[]
    for k in range(0,hilbertspace):
        for n in range(k+1,hilbertspace): 
            if(graph[k,n]!=0):
                constraint=np.zeros([hilbertspace,hilbertspace])
                constraint[n,k]=1
                constraint[k,n]=1
                constraints_op.append(qt.Qobj(constraint,dims=qt_dims)) 
    
elif(model==1): #Bell games
    J = np.array([
    [ 0.   ,  0.   ,  0.125,  0.125],
    [ 0.   ,  0.   ,  0.125, -0.125],
    [ 0.125,  0.125,  0.   ,  0.   ],
    [ 0.125, -0.125,  0.   ,  0.   ]])
    cost_op=qt.Qobj(J,dims=qt_dims)
    
    ##define operator for constraints
    constraints_op=[]
    for k in range(0,hilbertspace):
        constraint=np.zeros([hilbertspace,hilbertspace])
        constraint[k,k]=1
        constraints_op.append(qt.Qobj(constraint,dims=qt_dims)) 


n_constraints=len(constraints_op)#number of constraints
pauli_string_cost=[]
weights_pauli_cost=[]
pauli_string_constraints=[]
weights_pauli_constraints=[]

#get pauli strings and weights for cost function
pauli_list,weights=decomposePauli(cost_op)
pauli_string_cost=pauli_list
weights_pauli_cost=weights

#get pauli strings and weights for constraint
for k in range(n_constraints):
    pauli_list,weights=decomposePauli(constraints_op[k])
    pauli_string_constraints.append(pauli_list)
    weights_pauli_constraints.append(weights)

In [7]:
##get Pauli strings to generate ansatz space
#get all possible combinations of x Paulis
n_paulis=2**n_qubits
ansatz_pauli=np.zeros([n_paulis,n_qubits],dtype=int)
for k in range(n_paulis):
    ansatz_pauli[k,:]=numberToBase(k,2,n_qubits)

In [8]:
##solve classical SDP as reference
if(model==0): #Lovasz -theta number

    X = cp.Variable((hilbertspace,hilbertspace), symmetric=True)
    # Constraints
    # The operator >> denotes matrix inequality.
    J=np.ones([hilbertspace,hilbertspace])
    constraints = [X >> 0]##positive semidefinite
    constraints += [cp.trace(X) == 1] #trace 1 of density matrix
    for i in range(hilbertspace):
        for j in range(hilbertspace):
            if graph[i,j] == 1:
                constraints += [X[i,j] == 0]
    prob = cp.Problem(cp.Maximize(cp.trace(J@X)),
                      constraints)  
    theta = prob.solve(solver="CVXOPT",verbose=verbose_SDP)
    
    ##exact solution by classical SDP
    solution_exact = qt.Qobj(X.value,dims=qt_dims)
    print("")
    print("exact SDP value",np.trace(np.dot(J,X.value)))
elif(model==1): #Bell game

    J = np.array([
    [ 0.   ,  0.   ,  0.125,  0.125],
    [ 0.   ,  0.   ,  0.125, -0.125],
    [ 0.125,  0.125,  0.   ,  0.   ],
    [ 0.125, -0.125,  0.   ,  0.   ]])
    

    X = cp.Variable((hilbertspace,hilbertspace), symmetric=True)
    # Constraints
    # The operator >> denotes matrix inequality.
    constraints = [X >> 0]

    #constraint diagonals to be 1
    for i in range(hilbertspace):
        constraints += [X[i,i] == 1]
    prob = cp.Problem(cp.Maximize(cp.trace(J@X)),
                      constraints)  
    theta = prob.solve(solver="CVXOPT",verbose=verbose)
    solution_exact = qt.Qobj(X.value,dims=qt_dims)
    print("")
    print("exact SDP value",np.trace(np.dot(J,X.value)))

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) Jun 06 04:08:40 PM: Your problem has 64 variables, 26 constraints, and 0 parameters.
(CVXPY) Jun 06 04:08:40 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 06 04:08:40 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 06 04:08:40 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 06 04:08:40 PM: Compiling problem (target solver=CVXOPT).
(CVXPY) Jun 06 04:08:40 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> C

In [9]:
##generate initial_state that is used for NISQ SDP solver
if(ansatz_state==0):##product state all zero
    initial_state=qt.tensor([qt.basis(levels,0) for i in range(n_qubits)])
elif(ansatz_state==1):  #randomized circuit as initial state, consists of y rotations and CNOT gates
    depth=5 ##number of layers of circuit
    rand_angles=rng.random([depth,n_qubits])*2*np.pi #randomized angles
    rand_pauli=np.ones([depth,n_qubits],dtype=int)*2#y rotations

    entangling_gate_index=[[2*j,2*j+1] for j in range(n_qubits//2)]+[[2*j+1,2*j+2] for j in range((n_qubits-1)//2)]

    entangling_layer=prod([qt.qip.operations.cnot(n_qubits,j,k) for j,k in entangling_gate_index[::-1]])#need [::-1] to invert order so that unitaries are multiplied in correct order

    initial_state=qt.tensor([qt.basis(levels,0) for i in range(n_qubits)])

    ##do layered ansatz construction
    for j in range(depth):
        rot_op=[]
        for k in range(n_qubits):
            angle=rand_angles[j][k]
            if(rand_pauli[j][k]==1):
                rot_op.append(qt.qip.operations.rx(angle))
            elif(rand_pauli[j][k]==2):
                rot_op.append(qt.qip.operations.ry(angle))
            elif(rand_pauli[j][k]==3):
                rot_op.append(qt.qip.operations.rz(angle))


        initial_state=qt.tensor(rot_op)*initial_state

        initial_state=entangling_layer*initial_state

In [10]:
##generate ansatz space by using initial_state and applying  Pauli operators on it
expand_strings=ansatz_pauli[:n_ansatz]

n_ansatz_space=len(expand_strings)

expand_states=[]
for i in range(n_ansatz_space):
    expand_states.append(get_pauli_op(expand_strings[i])*initial_state)
    
    
    
E_matrix=np.zeros([n_ansatz_space,n_ansatz_space],dtype=np.complex128)

D_matrix=np.zeros([n_ansatz_space,n_ansatz_space],dtype=np.complex128)

constraint_matrix=np.zeros([n_constraints,n_ansatz_space,n_ansatz_space],dtype=np.complex128)


#calculate E, D and constraint matrix as overlaps of the ansatz space
#use the decomposed Pauli operators to measure the overlaps

for m in range(n_ansatz_space):
    for n in range(n_ansatz_space):
        E_matrix[m,n]=expand_states[m].overlap(expand_states[n])
        
        weights=weights_pauli_cost
        pauli_list=pauli_string_cost
        for j in range(len(pauli_list)):
            if(weights[j]!=0):
                D_matrix[m,n]+=weights[j]*(expand_states[m].overlap(get_pauli_op(pauli_list[j])*expand_states[n]))

        
        for k in range(n_constraints):
            weights=weights_pauli_constraints[k]
            pauli_list=pauli_string_constraints[k]

            for j in range(len(pauli_list)):
                if(weights[j]!=0):
                    constraint_matrix[k][m,n]+=weights[j]*(expand_states[m].overlap(get_pauli_op(pauli_list[j])*expand_states[n]))




In [11]:
#run classical part of NISQ SDP solver, solve SDP over measured overlap matrices
if(model==0): ##lovasz theta
    beta = cp.Variable((n_ansatz_space,n_ansatz_space), symmetric=True)

    # Constraints
    # The operator >> denotes matrix inequality.
    constraints = [beta >> 0] #positive semidefinite
    constraints += [cp.real(cp.trace(beta@E_matrix)) == 1] #trace 1 condition
    for k in range(n_constraints):
        if(np.sum(np.abs(constraint_matrix[k]))>0): #remove empty constraints as they can cause issues
            constraints += [cp.real(cp.trace(beta@constraint_matrix[k])) == 0]

    #initialise sdp sovlver for NISQ problem
    prob = cp.Problem(cp.Maximize(cp.real(cp.trace(beta@D_matrix))),constraints)  
    
    #run solver
    theta_NISQ = prob.solve(solver="CVXOPT",verbose=verbose_SDP)

    beta_NISQ = beta.value
    print("")
    print("SDP finished, cost function",np.trace(np.dot(D_matrix,beta_NISQ)))
    print("trace constraint",np.trace(np.dot(E_matrix,beta_NISQ)))
elif(model==1): #Bell games
    beta = cp.Variable((n_ansatz_space,n_ansatz_space), symmetric=True)
    # Constraints
    # The operator >> denotes matrix inequality.
    
    constraints = [beta >> 0]
    n_do_constraints=n_constraints
    if(n_ansatz_space==2):#
        n_do_constraints=n_do_constraints-1 #remove one constraint for 2 ansatz states as there is a bug with the solver that causes issues here
    for k in range(n_do_constraints):
        if(np.sum(np.abs(constraint_matrix[k]))>0): #remove empty D_matrix as they can cause issues
            constraints += [cp.real(cp.trace(beta@constraint_matrix[k])) == 1]

    prob = cp.Problem(cp.Maximize(cp.real(cp.trace(beta@D_matrix))),constraints)  
    theta_NISQ = prob.solve(solver="CVXOPT",verbose=verbose)

    beta_NISQ = beta.value
    print("")
    print("SDP finished",np.trace(np.dot(D_matrix,beta_NISQ)))

    
##hybrid density matrix as generated by NISQ SDP solver
NISQ_state=sum([sum([(expand_states[i]*expand_states[k].dag())*beta_NISQ[i,k] for i in range(n_ansatz_space)]) for k in range(n_ansatz_space)])



                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) Jun 06 04:08:41 PM: Your problem has 64 variables, 14 constraints, and 0 parameters.
(CVXPY) Jun 06 04:08:41 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 06 04:08:41 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 06 04:08:41 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 06 04:08:41 PM: Compiling problem (target solver=CVXOPT).
(CVXPY) Jun 06 04:08:41 PM: Reduction chain: Complex2Real -> FlipObjective -> Dcp2Cone -> Cvx

In [12]:
##evaluate error in regards to const function and constraints

##exact result from classical SDP for constraints and cost
exact_constraints=np.zeros(n_constraints)
exact_cost=np.real(qt.expect(cost_op,solution_exact))
for k in range(n_constraints):
    exact_constraints[k]=np.real(qt.expect(constraints_op[k],solution_exact))

##from NISQ SDP solver
NISQ_constraints=np.zeros(n_constraints)
NISQ_cost=np.real(qt.expect(cost_op,NISQ_state))
for k in range(n_constraints):
    NISQ_constraints[k]=np.real(qt.expect(constraints_op[k],NISQ_state))

#error between exact and NISQ solution
error_cost=np.abs(NISQ_cost-exact_cost)
error_constraints=np.sum([np.abs(NISQ_constraints[k]-exact_constraints[k]) for k in range(n_constraints)])

In [13]:
print("Total error between exact and NISQ SDP solution",error_cost+error_constraints)
print("Error cost",error_cost)
print("Error constraints",error_constraints)

Total error between exact and NISQ SDP solution 8.881784197001252e-16
Error cost 8.881784197001252e-16
Error constraints 0.0
